In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Susceptibility_to_Infections/GSE195434'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression correlates of HIV-1 susceptibility and gene expression changes following HIV-1 infection"
!Series_summary	"We sought to determine how gene expression changes during the first two years of HIV-1 infection among participants from HIV-1 serodiscordant couple cohorts from sub-Saharan Africa. This study included whole blood samples from 26 participants who did not have HIV-1 at study enrollment, had a steady sexual relationship with a partner with HIV-1 and acquired HIV-1 during follow-up. Most participants had samples from before and after infection."
!Series_overall_design	"Gene expression was measured by Illumina HumanHT-12 v4 Expression BeadChips. Whole blood samples were preserved at -80 degrees C in PAXgene tubes. Samples were not replicated on the arrays."
Sample Characteristics Dictionary:
{0: ['individual: PTID 1', 'individual: PTID 4', 'individual: PTID 6', 'individual: PTID 20', 'individual: PTID 9', 'individual: PTID 7', 'in

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # It's stated that the data is measured by Illumina HumanHT-12 v4 Expression BeadChips

# Check availability of 'Susceptibility_to_Infections'
trait_row = 1  # This key likely corresponds to infection status with values 'Post.infection' and 'Pre.infection'

# There's no explicit 'age' or 'gender' data available.
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    # Assuming infection status is binary where 'Pre.infection' indicates no infection (0) and 'Post.infection' indicates infection (1)
    val = value.split(':')[1].strip()
    if val == 'Pre.infection':
        return 0
    elif val == 'Post.infection':
        return 1
    return None

def convert_age(value):
    return None  # No data available

def convert_gender(value):
    return None  # No data available

# Save cohort information
save_cohort_info('GSE195434', './preprocessed/Susceptibility_to_Infections/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Susceptibility_to_Infections', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Susceptibility_to_Infections/trait_data/GSE195434.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5836645': [1], 'GSM5836646': [1], 'GSM5836647': [1], 'GSM5836648': [1], 'GSM5836649': [1], 'GSM5836650': [1], 'GSM5836651': [1], 'GSM5836652': [1], 'GSM5836653': [0], 'GSM5836654': [1], 'GSM5836655': [1], 'GSM5836656': [1], 'GSM5836657': [1], 'GSM5836658': [1], 'GSM5836659': [1], 'GSM5836660': [1], 'GSM5836661': [1], 'GSM5836662': [0], 'GSM5836663': [0], 'GSM5836664': [1], 'GSM5836665': [1], 'GSM5836666': [1], 'GSM5836667': [1], 'GSM5836668': [1], 'GSM5836669': [1], 'GSM5836670': [1], 'GSM5836671': [1], 'GSM5836672': [1], 'GSM5836673': [1], 'GSM5836674': [0], 'GSM5836675': [1], 'GSM5836676': [1], 'GSM5836677': [1], 'GSM5836678': [1], 'GSM5836679': [1], 'GSM5836680': [1], 'GSM5836681': [1], 'GSM5836682': [1], 'GSM5836683': [1], 'GSM5836684': [1], 'GSM5836685': [1], 'GSM5836686': [1], 'GSM5836687': [1], 'GSM5836688': [1], 'GSM5836689': [1], 'GSM5836690': [1], 'GSM5836691': [1], 'GSM5836692': [1], 'GSM5836693': [1], 'GSM5836694': [1], 'GSM5836695': [1], 'GSM5836696': [0], 'GSM5836697

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [7]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # It's stated that the data is measured by Illumina HumanHT-12 v4 Expression BeadChips

# Check availability of 'Susceptibility_to_Infections'
trait_row = 1  # This key likely corresponds to infection status with values 'Post.infection' and 'Pre.infection'

# There's no explicit 'age' or 'gender' data available.
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    # Assuming infection status is binary where 'Pre.infection' indicates no infection (0) and 'Post.infection' indicates infection (1)
    val = value.split(':')[1].strip()
    if val == 'Pre.infection':
        return 0
    elif val == 'Post.infection':
        return 1
    return None

def convert_age(value):
    return None  # No data available

def convert_gender(value):
    return None  # No data available

# Save cohort information
save_cohort_info('GSE195434', './preprocessed/Susceptibility_to_Infections/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Susceptibility_to_Infections', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Susceptibility_to_Infections/trait_data/GSE195434.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5836645': [1], 'GSM5836646': [1], 'GSM5836647': [1], 'GSM5836648': [1], 'GSM5836649': [1], 'GSM5836650': [1], 'GSM5836651': [1], 'GSM5836652': [1], 'GSM5836653': [0], 'GSM5836654': [1], 'GSM5836655': [1], 'GSM5836656': [1], 'GSM5836657': [1], 'GSM5836658': [1], 'GSM5836659': [1], 'GSM5836660': [1], 'GSM5836661': [1], 'GSM5836662': [0], 'GSM5836663': [0], 'GSM5836664': [1], 'GSM5836665': [1], 'GSM5836666': [1], 'GSM5836667': [1], 'GSM5836668': [1], 'GSM5836669': [1], 'GSM5836670': [1], 'GSM5836671': [1], 'GSM5836672': [1], 'GSM5836673': [1], 'GSM5836674': [0], 'GSM5836675': [1], 'GSM5836676': [1], 'GSM5836677': [1], 'GSM5836678': [1], 'GSM5836679': [1], 'GSM5836680': [1], 'GSM5836681': [1], 'GSM5836682': [1], 'GSM5836683': [1], 'GSM5836684': [1], 'GSM5836685': [1], 'GSM5836686': [1], 'GSM5836687': [1], 'GSM5836688': [1], 'GSM5836689': [1], 'GSM5836690': [1], 'GSM5836691': [1], 'GSM5836692': [1], 'GSM5836693': [1], 'GSM5836694': [1], 'GSM5836695': [1], 'GSM5836696': [0], 'GSM5836697

### Step 3: Gene Data Extraction

In [8]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 5: Gene Annotation (Conditional)

In [9]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [10]:
# 1. Identify the key storing identifiers and the key storing gene symbols in gene annotation data.
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and rename the resulting gene expression dataframe to "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [11]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Susceptibility_to_Infections/gene_data/GSE195434.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Susceptibility_to_Infections')

# 4. Save the cohort information.
save_cohort_info('GSE195434', './preprocessed/Susceptibility_to_Infections/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Susceptibility_to_Infections/GSE195434.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Susceptibility_to_Infections', the least common label is '0.0' with 21 occurrences. This represents 23.33% of the dataset.
The distribution of the feature 'Susceptibility_to_Infections' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Susceptibility_to_Infections/cohort_info.json
